In [ ]:
!pip install spacy nltk pandas tqdm inflect
!python -m spacy download en_core_web_sm
import nltk
nltk.download('wordnet')
nltk.download('omw-1.4')

In [ ]:
import os
import json
import random
import re
from pathlib import Path
import pandas as pd
import spacy
from nltk.corpus import wordnet as wn
from tqdm import tqdm
nlp = spacy.load('en_core_web_sm')
import inflect
p = inflect.engine()

In [ ]:
RUSSIAN_COMPLEX_PATTERNS = {
    'premises': {
        'all': [
            {'tpl': 'Все {a_pl} являются {b_pl}.', 'a_type': 'plural', 'b_type': 'plural'},
            {'tpl': 'Каждый из {a_pl} принадлежит к {b_pl}.', 'a_type': 'plural', 'b_type': 'plural'},
            {'tpl': 'Без исключений все {a_pl} — {b_pl}.', 'a_type': 'plural', 'b_type': 'plural'},
            {'tpl': 'Класс {a_pl} полностью входит в {b_pl}.', 'a_type': 'plural', 'b_type': 'plural'},
            {'tpl': 'Все представители {a_pl} относятся к {b_pl}.', 'a_type': 'plural', 'b_type': 'plural'},
            {'tpl': 'Любой объект, являющийся {a_sg_art}, также является {b_sg_art}.', 'a_type': 'singular_art', 'b_type': 'singular_art'},
            {'tpl': 'Если нечто есть {a_sg_art}, то оно обязательно и {b_sg_art}.', 'a_type': 'singular_art', 'b_type': 'singular_art'},
            {'tpl': 'Все без исключения {a_pl} суть {b_pl}.', 'a_type': 'plural', 'b_type': 'plural'},
            {'tpl': 'Для каждого {a_sg} верно, что он является {b_sg_art}.', 'a_type': 'singular', 'b_type': 'singular_art'},
            {'tpl': 'Множество {a_pl} является подмножеством {b_pl}.', 'a_type': 'plural', 'b_type': 'plural'},
            {'tpl': 'Каждое {a_sg} по определению относится к {b_pl}.', 'a_type': 'singular', 'b_type': 'plural'},
            {'tpl': 'Не существует таких {a_pl}, которые не были бы {b_pl}.', 'a_type': 'plural', 'b_type': 'plural'},
            {'tpl': 'Любое {a_sg} попадает в категорию {b_pl}.', 'a_type': 'singular', 'b_type': 'plural'},
            {'tpl': 'Совокупность всех {a_pl} включена в класс {b_pl}.', 'a_type': 'plural', 'b_type': 'plural'},
            {'tpl': 'Всякий {a_sg} рассматривается как {b_sg_art}.', 'a_type': 'singular', 'b_type': 'singular_art'}
        ],
        'some': [
            {'tpl': 'Некоторые {a_pl} являются {b_pl}.', 'a_type': 'plural', 'b_type': 'plural'},
            {'tpl': 'Есть {a_pl}, которые {b_pl}.', 'a_type': 'plural', 'b_type': 'plural'},
            {'tpl': 'Часть {a_pl} относится к {b_pl}.', 'a_type': 'plural', 'b_type': 'plural'},
            {'tpl': 'Небольшая доля {a_pl} — это {b_pl}.', 'a_type': 'plural', 'b_type': 'plural'},
            {'tpl': 'Среди {a_pl} встречаются те, кто {b_pl}.', 'a_type': 'plural', 'b_type': 'plural'},
            {'tpl': 'Существует по крайней мере один {a_sg}, который является {b_sg}.', 'a_type': 'singular', 'b_type': 'singular'},
            {'tpl': 'Кое-какие {a_pl} могут считаться {b_pl}.', 'a_type': 'plural', 'b_type': 'plural'},
            {'tpl': 'Обнаружены {a_pl}, обладающие свойствами {b_pl}.', 'a_type': 'plural', 'b_type': 'plural'},
            {'tpl': 'Не все, но некоторые {a_pl} — это {b_pl}.', 'a_type': 'plural', 'b_type': 'plural'},
            {'tpl': 'Имеются примеры {a_pl}, которые также являются {b_pl}.', 'a_type': 'plural', 'b_type': 'plural'},
            {'tpl': 'В ряде случаев {a_sg_art} является {b_sg_art}.', 'a_type': 'singular_art', 'b_type': 'singular_art'},
            {'tpl': 'Определенная часть {a_pl} входит в состав {b_pl}.', 'a_type': 'plural', 'b_type': 'plural'},
            {'tpl': 'Бывает, что {a_sg} оказывается {b_sg_art}.', 'a_type': 'singular', 'b_type': 'singular_art'},
            {'tpl': 'Можно найти такие {a_pl}, которые относятся к {b_pl}.', 'a_type': 'plural', 'b_type': 'plural'},
            {'tpl': 'Отдельные представители {a_pl} являются {b_pl}.', 'a_type': 'plural', 'b_type': 'plural'}
        ],
        'no': [
            {'tpl': 'Ни один из {a_pl} не является {b_pl}.', 'a_type': 'plural', 'b_type': 'plural'},
            {'tpl': 'Нет {a_pl}, которые были бы {b_pl}.', 'a_type': 'plural', 'b_type': 'plural'},
            {'tpl': '{a_pl} и {b_pl} не пересекаются.', 'a_type': 'plural', 'b_type': 'plural'},
            {'tpl': 'Класс {a_pl} не содержит {b_pl}.', 'a_type': 'plural', 'b_type': 'plural'},
            {'tpl': 'Абсолютно никакие {a_pl} не относятся к {b_pl}.', 'a_type': 'plural', 'b_type': 'plural'},
            {'tpl': 'Не существует {a_sg}, который мог бы считаться {b_sg_art}.', 'a_type': 'singular', 'b_type': 'singular_art'},
            {'tpl': 'Никакое {a_sg_art} не может быть {b_sg_art}.', 'a_type': 'singular_art', 'b_type': 'singular_art'},
            {'tpl': 'Полностью исключено, что {a_pl} являются {b_pl}.', 'a_type': 'plural', 'b_type': 'plural'},
            {'tpl': 'Ни один представитель {a_pl} не входит в {b_pl}.', 'a_type': 'plural', 'b_type': 'plural'},
            {'tpl': 'Между {a_pl} и {b_pl} нет ничего общего.', 'a_type': 'plural', 'b_type': 'plural'},
            {'tpl': 'Никакой {a_sg} никогда не станет {b_sg_art}.', 'a_type': 'singular', 'b_type': 'singular_art'},
            {'tpl': 'Категории {a_pl} и {b_pl} взаимоисключаемы.', 'a_type': 'plural', 'b_type': 'plural'},
            {'tpl': 'В группе {a_pl} нельзя встретить ни одного {b_sg}.', 'a_type': 'plural', 'b_type': 'singular'},
            {'tpl': 'Если нечто является {a_sg_art}, оно не может быть {b_sg_art}.', 'a_type': 'singular_art', 'b_type': 'singular_art'},
            {'tpl': 'Для всех {a_pl} верно, что они не {b_pl}.', 'a_type': 'plural', 'b_type': 'plural'}
        ],
        'some_not': [
            {'tpl': 'Некоторые {a_pl} не являются {b_pl}.', 'a_type': 'plural', 'b_type': 'plural'},
            {'tpl': 'Существуют {a_pl}, которые не {b_pl}.', 'a_type': 'plural', 'b_type': 'plural'},
            {'tpl': 'Часть {a_pl} не принадлежит к {b_pl}.', 'a_type': 'plural', 'b_type': 'plural'},
            {'tpl': 'Не все {a_pl} являются {b_pl}.', 'a_type': 'plural', 'b_type': 'plural'},
            {'tpl': 'Среди {a_pl} есть те, кто не {b_pl}.', 'a_type': 'plural', 'b_type': 'plural'},
            {'tpl': 'По крайней мере один {a_sg} не относится к {b_pl}.', 'a_type': 'singular', 'b_type': 'plural'},
            {'tpl': 'Имеются {a_pl}, лишенные свойств {b_pl}.', 'a_type': 'plural', 'b_type': 'plural'},
            {'tpl': 'Не каждое {a_sg} можно назвать {b_sg_art}.', 'a_type': 'singular', 'b_type': 'singular_art'},
            {'tpl': 'Отдельные {a_pl} находятся вне категории {b_pl}.', 'a_type': 'plural', 'b_type': 'plural'},
            {'tpl': 'Определенное количество {a_pl} не суть {b_pl}.', 'a_type': 'plural', 'b_type': 'plural'},
            {'tpl': 'Можно указать на {a_pl}, которые не являются {b_pl}.', 'a_type': 'plural', 'b_type': 'plural'},
            {'tpl': 'Для части {a_pl} неверно, что они {b_pl}.', 'a_type': 'plural', 'b_type': 'plural'},
            {'tpl': 'Бывает, что {a_sg_art} не является {b_sg_art}.', 'a_type': 'singular_art', 'b_type': 'singular_art'},
            {'tpl': 'Кое-какие из {a_pl} не входят в класс {b_pl}.', 'a_type': 'plural', 'b_type': 'plural'},
            {'tpl': 'В ряде случаев {a_sg} не принадлежит к {b_pl}.', 'a_type': 'singular', 'b_type': 'plural'}
        ]
    },
    'conclusions': [
        'Следовательно, {conclusion}.',
        'Отсюда следует, что {conclusion}.',
        'Таким образом, {conclusion}.',
        'В результате получается, что {conclusion}.',
        'Итак, {conclusion}.',
        'Из этого мы можем заключить, что {conclusion}.',
        'По этой причине {conclusion}.',
        'Это приводит нас к выводу, что {conclusion}.',
        'Логично предположить, что {conclusion}.',
        'Значит, {conclusion}.'
    ]
}

In [ ]:
RUSSIAN_COMPLEX_PATTERNS_WITH_FOURTH_VAR = {
    'premises': {
        'all': [
            {'tpl': 'Каждое {d_sg}, которое является {a_sg_art}, также является {b_sg_art}.', 'a_type': 'singular_art', 'b_type': 'singular_art', 'd_type': 'singular'},
            {'tpl': 'Если {d_sg_art} — это {a_sg_art}, то оно {b_sg_art}.', 'a_type': 'singular_art', 'b_type': 'singular_art', 'd_type': 'singular_art'},
            {'tpl': 'Все {d_pl}, являющиеся {a_pl}, также {b_pl}.', 'a_type': 'plural', 'b_type': 'plural', 'd_type': 'plural'},
            {'tpl': 'Без исключения: каждый {d_sg} с признаком {a_sg} — {b_sg}.', 'a_type': 'singular', 'b_type': 'singular', 'd_type': 'singular'},
            {'tpl': 'Для каждого {d_sg}, являющегося {a_sg_art}, верно, что оно {b_sg_art}.', 'a_type': 'singular_art', 'b_type': 'singular_art', 'd_type': 'singular'},
            {'tpl': 'Любой объект из {d_pl}, если он относится к {a_pl}, является {b_pl}.', 'a_type': 'plural', 'b_type': 'plural', 'd_type': 'plural'},
            {'tpl': 'Всякий представитель {d_pl}, обладающий качеством {a_sg}, есть {b_sg}.', 'a_type': 'singular', 'b_type': 'singular', 'd_type': 'plural'},
            {'tpl': 'В группе {d_pl} все {a_pl} одновременно являются {b_pl}.', 'a_type': 'plural', 'b_type': 'plural', 'd_type': 'plural'},
            {'tpl': 'Если {d_sg} есть и {a_sg}, то оно непременно {b_sg}.', 'a_type': 'singular', 'b_type': 'singular', 'd_type': 'singular'},
            {'tpl': 'Все те {d_pl}, что считаются {a_pl}, признаются и {b_pl}.', 'a_type': 'plural', 'b_type': 'plural', 'd_type': 'plural'},
            {'tpl': 'Каждое {d_sg} из категории {a_pl} обязательно относится к {b_pl}.', 'a_type': 'plural', 'b_type': 'plural', 'd_type': 'singular'},
            {'tpl': 'Для любого {d_sg}, если он {a_sg_art}, истинно, что он {b_sg_art}.', 'a_type': 'singular_art', 'b_type': 'singular_art', 'd_type': 'singular'},
            {'tpl': 'Множество {d_pl}, входящих в {a_pl}, целиком содержится в {b_pl}.', 'a_type': 'plural', 'b_type': 'plural', 'd_type': 'plural'},
            {'tpl': 'Ни один {d_sg} не может быть {a_sg_art}, не являясь при этом {b_sg_art}.', 'a_type': 'singular_art', 'b_type': 'singular_art', 'd_type': 'singular'},
            {'tpl': 'В классе {d_pl} всё, что есть {a_sg_art}, является также {b_sg_art}.', 'a_type': 'singular_art', 'b_type': 'singular_art', 'd_type': 'plural'}
        ],
        'some': [
            {'tpl': 'Существует по крайней мере один {d_sg}, который одновременно {a_sg} и {b_sg}.', 'a_type': 'singular', 'b_type': 'singular', 'd_type': 'singular'},
            {'tpl': 'Некоторые {d_pl}, являющиеся {a_pl}, являются {b_pl}.', 'a_type': 'plural', 'b_type': 'plural', 'd_type': 'plural'},
            {'tpl': 'Часть {d_pl}, которые {a_pl}, относится к {b_pl}.', 'a_type': 'plural', 'b_type': 'plural', 'd_type': 'plural'},
            {'tpl': 'Есть {d_pl}, которые {a_pl} и одновременно {b_pl}.', 'a_type': 'plural', 'b_type': 'plural', 'd_type': 'plural'},
            {'tpl': 'Хотя бы один {d_sg}, являющийся {a_sg_art}, является {b_sg_art}.', 'a_type': 'singular_art', 'b_type': 'singular_art', 'd_type': 'singular'},
            {'tpl': 'Кое-какие {d_pl} совмещают в себе признаки {a_pl} и {b_pl}.', 'a_type': 'plural', 'b_type': 'plural', 'd_type': 'plural'},
            {'tpl': 'Обнаружен {d_sg}, который входит как в {a_pl}, так и в {b_pl}.', 'a_type': 'plural', 'b_type': 'plural', 'd_type': 'singular'},
            {'tpl': 'Среди {d_pl} встречаются объекты, являющиеся и {a_pl}, и {b_pl}.', 'a_type': 'plural', 'b_type': 'plural', 'd_type': 'plural'},
            {'tpl': 'Некоторое количество {d_pl} классифицируется как {a_pl} и {b_pl}.', 'a_type': 'plural', 'b_type': 'plural', 'd_type': 'plural'},
            {'tpl': 'Для определенных {d_pl} верно, что они и {a_pl}, и {b_pl}.', 'a_type': 'plural', 'b_type': 'plural', 'd_type': 'plural'},
            {'tpl': 'Имеется случай, когда {d_sg} есть {a_sg_art} и {b_sg_art}.', 'a_type': 'singular_art', 'b_type': 'singular_art', 'd_type': 'singular'},
            {'tpl': 'В ряде ситуаций {d_sg} может рассматриваться и как {a_sg_art}, и как {b_sg_art}.', 'a_type': 'singular_art', 'b_type': 'singular_art', 'd_type': 'singular'},
            {'tpl': 'Существуют примеры {d_pl}, принадлежащих одновременно к {a_pl} и к {b_pl}.', 'a_type': 'plural', 'b_type': 'plural', 'd_type': 'plural'},
            {'tpl': 'Отдельные {d_pl} обладают одновременно качествами {a_pl} и {b_pl}.', 'a_type': 'plural', 'b_type': 'plural', 'd_type': 'plural'},
            {'tpl': 'Можно найти такой {d_sg}, который будет и {a_sg}, и {b_sg}.', 'a_type': 'singular', 'b_type': 'singular', 'd_type': 'singular'}
        ],
        'no': [
            {'tpl': 'Ни один из {d_pl}, которые {a_pl}, не является {b_pl}.', 'a_type': 'plural', 'b_type': 'plural', 'd_type': 'plural'},
            {'tpl': 'Не существует {d_sg}, который был бы одновременно {a_sg} и {b_sg}.', 'a_type': 'singular', 'b_type': 'singular', 'd_type': 'singular'},
            {'tpl': 'Класс {d_pl}, являющийся {a_pl}, не содержит {b_pl}.', 'a_type': 'plural', 'b_type': 'plural', 'd_type': 'plural'},
            {'tpl': 'Невозможно, чтобы {d_sg} был одновременно {a_sg} и {b_sg}.', 'a_type': 'singular', 'b_type': 'singular', 'd_type': 'singular'},
            {'tpl': 'Ни одно {d_sg}, являющееся {a_sg}, не является {b_sg}.', 'a_type': 'singular', 'b_type': 'singular', 'd_type': 'singular'},
            {'tpl': 'В классе {d_pl} нет объектов, которые были бы и {a_pl}, и {b_pl}.', 'a_type': 'plural', 'b_type': 'plural', 'd_type': 'plural'},
            {'tpl': 'Никакой представитель {d_pl} не может совмещать {a_pl} с {b_pl}.', 'a_type': 'plural', 'b_type': 'plural', 'd_type': 'plural'},
            {'tpl': 'Для всех {d_pl} исключено одновременное наличие признаков {a_pl} и {b_pl}.', 'a_type': 'plural', 'b_type': 'plural', 'd_type': 'plural'},
            {'tpl': 'Если нечто есть {d_sg} и {a_sg}, оно никак не может быть {b_sg}.', 'a_type': 'singular', 'b_type': 'singular', 'd_type': 'singular'},
            {'tpl': 'Среди {d_pl} не найдено ни одного примера {a_pl}, являющегося {b_pl}.', 'a_type': 'plural', 'b_type': 'plural', 'd_type': 'plural'},
            {'tpl': 'Категория {d_pl} полностью исключает пересечение {a_pl} и {b_pl}.', 'a_type': 'plural', 'b_type': 'plural', 'd_type': 'plural'},
            {'tpl': 'Никакое {d_sg}, обладающее свойством {a_sg}, не обладает свойством {b_sg}.', 'a_type': 'singular', 'b_type': 'singular', 'd_type': 'singular'},
            {'tpl': 'Для любого {d_sg} неверно, что он и {a_sg_art}, и {b_sg_art}.', 'a_type': 'singular_art', 'b_type': 'singular_art', 'd_type': 'singular'},
            {'tpl': 'В пределах множества {d_pl} ни одно {a_sg_art} не является {b_sg_art}.', 'a_type': 'singular_art', 'b_type': 'singular_art', 'd_type': 'plural'},
            {'tpl': 'Объекты {d_pl}, являющиеся {a_pl}, лишены признака {b_pl}.', 'a_type': 'plural', 'b_type': 'plural', 'd_type': 'plural'}
        ],
        'some_not': [
            {'tpl': 'Не каждый {d_sg}, который {a_sg_art}, является {b_sg_art}.', 'a_type': 'singular_art', 'b_type': 'singular_art', 'd_type': 'singular'},
            {'tpl': 'Существует по крайней мере один {d_sg}, который {a_sg}, но не {b_sg}.', 'a_type': 'singular', 'b_type': 'singular', 'd_type': 'singular'},
            {'tpl': 'Некоторые {d_pl}, которые {a_pl}, не являются {b_pl}.', 'a_type': 'plural', 'b_type': 'plural', 'd_type': 'plural'},
            {'tpl': 'Часть {d_pl} с признаком {a_pl} исключена из {b_pl}.', 'a_type': 'plural', 'b_type': 'plural', 'd_type': 'plural'},
            {'tpl': 'Есть {d_pl}, которые {a_pl} и тем не менее не {b_pl}.', 'a_type': 'plural', 'b_type': 'plural', 'd_type': 'plural'},
            {'tpl': 'Обнаружены {d_pl}, относящиеся к {a_pl}, но не к {b_pl}.', 'a_type': 'plural', 'b_type': 'plural', 'd_type': 'plural'},
            {'tpl': 'Хотя бы один {d_sg} является {a_sg_art}, не будучи при этом {b_sg_art}.', 'a_type': 'singular_art', 'b_type': 'singular_art', 'd_type': 'singular'},
            {'tpl': 'Для определенных {d_pl} из класса {a_pl} неверно, что они {b_pl}.', 'a_type': 'plural', 'b_type': 'plural', 'd_type': 'plural'},
            {'tpl': 'Не для всех {d_pl}, входящих в {a_pl}, справедливо утверждение, что они {b_pl}.', 'a_type': 'plural', 'b_type': 'plural', 'd_type': 'plural'},
            {'tpl': 'Имеются представители {d_pl} со свойствами {a_pl}, лишенные признаков {b_pl}.', 'a_type': 'plural', 'b_type': 'plural', 'd_type': 'plural'},
            {'tpl': 'Бывает, что {d_sg}, являясь {a_sg_art}, не является {b_sg_art}.', 'a_type': 'singular_art', 'b_type': 'singular_art', 'd_type': 'singular'},
            {'tpl': 'В группе {d_pl} можно найти {a_pl}, не являющиеся {b_pl}.', 'a_type': 'plural', 'b_type': 'plural', 'd_type': 'plural'},
            {'tpl': 'Отдельные {d_pl}, принадлежащие к {a_pl}, не входят в состав {b_pl}.', 'a_type': 'plural', 'b_type': 'plural', 'd_type': 'plural'},
            {'tpl': 'Кое-какие {d_pl} обладают признаком {a_pl}, но не признаком {b_pl}.', 'a_type': 'plural', 'b_type': 'plural', 'd_type': 'plural'},
            {'tpl': 'Среди объектов {d_pl} есть те, кто {a_pl}, но не {b_pl}.', 'a_type': 'plural', 'b_type': 'plural', 'd_type': 'plural'}
        ]
    }
}

In [ ]:
def get_singular_and_plural(term):
    term = term.strip()
    sg_attempt = p.singular_noun(term)
    if sg_attempt is not False:
        sg = sg_attempt
        pl = term
    else:
        sg = term
        pl = p.plural_noun(term)
    if sg is False:
        sg = term
    if not pl:
        pl = term
    return sg, pl

In [ ]:
def get_article(term):
    term = term.strip()
    return p.an(term)

In [ ]:
def parse_sentence_for_type_and_terms(sentence):
    sentence = sentence.strip()
    match = re.match(r'All (.*) that are (.*) are (.*)\.', sentence)
    if match: return 'all', match.group(1), match.group(2), match.group(3)
    match = re.match(r'Some (.*) that are (.*) are not (.*)\.', sentence)
    if match: return 'some_not', match.group(1), match.group(2), match.group(3)
    match = re.match(r'Some (.*) that are (.*) are (.*)\.', sentence)
    if match: return 'some', match.group(1), match.group(2), match.group(3)
    match = re.match(r'No (.*) that are (.*) are (.*)\.', sentence)
    if match: return 'no', match.group(1), match.group(2), match.group(3)
    match = re.match(r'All (.*) are (.*)\.', sentence)
    if match: return 'all', None, match.group(1), match.group(2)
    match = re.match(r'Some (.*) are not (.*)\.', sentence)
    if match: return 'some_not', None, match.group(1), match.group(2)
    match = re.match(r'Some (.*) are (.*)\.', sentence)
    if match: return 'some', None, match.group(1), match.group(2)
    match = re.match(r'No (.*) are (.*)\.', sentence)
    if match: return 'no', None, match.group(1), match.group(2)
    raise ValueError(f'Could not parse sentence: {sentence}')

In [ ]:
def create_sub_dict(term_original_subject, term_original_predicate, term_original_fourth=None):
    sub_map = {}
    sg_a, pl_a = get_singular_and_plural(term_original_subject)
    sub_map['a_sg'] = sg_a
    sub_map['a_pl'] = pl_a
    sub_map['a_sg_art'] = get_article(sg_a)
    sg_b, pl_b = get_singular_and_plural(term_original_predicate)
    sub_map['b_sg'] = sg_b
    sub_map['b_pl'] = pl_b
    sub_map['b_sg_art'] = get_article(sg_b)
    if term_original_fourth:
        sg_d, pl_d = get_singular_and_plural(term_original_fourth)
        sub_map['d_sg'] = sg_d
        sub_map['d_pl'] = pl_d
        sub_map['d_sg_art'] = get_article(sg_d)
    return sub_map

In [ ]:
def safe_lowercase_first(s, sub_map):
    for val in sub_map.values():
        if s.startswith(val):
            return s
    return s[:1].lower() + s[1:]

In [ ]:
def convert_syllogism_to_complex(syllogism_data, complex_patterns_data, complex_patterns_data_with_fourth_var):
    simple_syllogism = syllogism_data['syllogism']
    sentences = [s.strip() for s in simple_syllogism.split('.') if s.strip()]
    sentences = [s + '.' for s in sentences]
    premises_simple = sentences[:-1]
    conclusion_simple = sentences[-1]
    complex_premises = []
    for premise in premises_simple:
        p_type, p_fourth, p_subject, p_predicate = parse_sentence_for_type_and_terms(premise)
        sub_map = create_sub_dict(p_subject, p_predicate, p_fourth)
        if p_fourth:
            p_template = random.choice(complex_patterns_data_with_fourth_var['premises'][p_type])
        else:
            p_template = random.choice(complex_patterns_data['premises'][p_type])
        p_complex = p_template['tpl'].format(**sub_map)
        p_complex = p_complex[0].upper() + p_complex[1:]
        complex_premises.append(p_complex)
    c_type, c_fourth, c_subject, c_predicate = parse_sentence_for_type_and_terms(conclusion_simple)
    c_sub_map = create_sub_dict(c_subject, c_predicate, c_fourth)
    if c_fourth:
        c_premise_template = random.choice(complex_patterns_data_with_fourth_var['premises'][c_type])
    else:
        c_premise_template = random.choice(complex_patterns_data['premises'][c_type])
    c_premise_complex = c_premise_template['tpl'].format(**c_sub_map)
    c_premise_complex = safe_lowercase_first(c_premise_complex, c_sub_map)
    c_wrapper_template = random.choice(complex_patterns_data['conclusions'])
    c_complex = c_wrapper_template.format(conclusion=c_premise_complex)
    c_complex = c_complex[:-1] if c_complex.endswith('.') else c_complex
    c_complex = c_complex[0].upper() + c_complex[1:]
    final_complex_syllogism = ' '.join(complex_premises) + ' ' + c_complex
    new_data = syllogism_data.copy()
    new_data['syllogism'] = final_complex_syllogism
    return new_data

In [ ]:
def process_dataset(input_data, complex_patterns_data, complex_patterns_data_with_fourth_var):
    complex_dataset = []
    for item in input_data:
        try:
            complex_item = convert_syllogism_to_complex(item, complex_patterns_data, complex_patterns_data_with_fourth_var)
            complex_dataset.append(complex_item)
        except ValueError as e:
            print(f" Skipping item ID {item.get('id', 'N/A')} due to parsing error: {e}")
            continue
    return complex_dataset

In [ ]:
# Example data and run (sanity check)
sample_data = [
    {'id': 'r1', 'syllogism': 'All cats are animals. Some pets are cats. Some pets are animals.', 'validity': True},
    {'id': 'r2', 'syllogism': 'No cars are birds. Some vehicles are cars. Some vehicles are not birds.', 'validity': True}
]
complex_syllogism_data = process_dataset(sample_data, RUSSIAN_COMPLEX_PATTERNS, RUSSIAN_COMPLEX_PATTERNS_WITH_FOURTH_VAR)
print('
--- Converted Russian Complex Syllogisms ---')
print(json.dumps(complex_syllogism_data, indent=4, ensure_ascii=False))